In [155]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
from prophet import Prophet

from sklearn.metrics import mean_squared_error, accuracy_score, f1_score, roc_auc_score,  precision_score, recall_score
from sklearn import preprocessing
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import TimeSeriesSplit
import itertools

import os
os.chdir('..')

os.chdir('/Users/marcbr/Documents/gitrepos/604Final/')
import gc
import sys

import mlforecast


import datetime
from utils import utils, models

In [96]:
os.chdir('/Users/marcbr/Documents/gitrepos/604Final/')

In [97]:
daily_df = pd.read_csv('data/daily_data.csv')

In [17]:
daily_df.tail()

,station,date,temp_max,temp_mean,temp_min,rainfall,snow
76667,PALH0,2023-11-18,-6.7,-10.191667,-13.3,False,False
76668,PALH0,2023-11-19,-10.6,-13.812500,-16.1,False,False
76669,PALH0,2023-11-20,-10.6,-16.162500,-18.9,False,False
76670,PALH0,2023-11-21,-4.4,-9.512500,-12.8,False,False
76671,PALH0,2023-11-22,-3.9,-4.911765,-6.4,True,True


In [8]:
# test_date = str(datetime.date.today() - datetime.timedelta(days=4))
test_date = '2023-11-16'
daily = True
daily_df["date"] = pd.to_datetime(daily_df["date"])
train_data_pd, test_data_pd = models.train_test_split(daily_df, test_date, daily)

# Prophet training

### First for one station 

In [56]:
train_1 = train_data_pd.loc[train_data_pd.station == '72202']
test_1 = test_data_pd.loc[test_data_pd.station == '72202']

In [57]:
train_1 = pd.DataFrame(train_1.rename(columns={'date': 'ds', 'temp_min': 'y'}))
test_1 = pd.DataFrame(test_1.rename(columns={'date': 'ds', 'temp_min': 'y'}))

### Adding regressors

In [26]:
model = Prophet()

In [27]:
train_1.columns

Index(['station', 'ds', 'temp_max', 'temp_mean', 'y', 'rainfall', 'snow'], dtype='object')

In [28]:
model.add_regressor(name='temp_max')
model.add_regressor(name='temp_mean')
model.add_regressor(name='rainfall')
model.add_regressor(name='snow')

In [29]:
model.fit(train_1[['ds', 'y', 'temp_max', 'temp_mean', 'rainfall', 'snow']])

10:27:27 - cmdstanpy - INFO - Chain [1] start processing
10:27:27 - cmdstanpy - INFO - Chain [1] done processing


In [31]:
future = model.make_future_dataframe(periods=7)
future = future.merge(test_1[['ds', 'temp_max', 'temp_mean', 'rainfall', 'snow']], on='ds') 

In [32]:
forecast_1 = model.predict(future)

In [33]:
eval = (pd.DataFrame(test_1[['ds', 'y']])
              .merge(forecast_1[['ds', 'yhat']], on='ds')
              )

In [35]:
eval.head()

,ds,y,yhat
0,2023-11-16,21.0,20.864130
1,2023-11-17,23.0,22.177245
2,2023-11-18,22.0,21.341177
3,2023-11-19,21.0,20.522439
4,2023-11-20,20.6,21.985756


In [37]:
np.sqrt(mean_squared_error(eval.y, eval.yhat))

0.7565998817695939

#### Avg temp

In [58]:
train_1 = pd.DataFrame(train_1.rename(columns={'date': 'ds', 'y': 'temp_min'})).rename(columns={'temp_mean': 'y'})
test_1 = pd.DataFrame(test_1.rename(columns={'date': 'ds', 'y': 'temp_min'})).rename(columns={'temp_mean': 'y'})

In [62]:
model = Prophet()
model.add_regressor(name='temp_max')
model.add_regressor(name='temp_min')
model.add_regressor(name='rainfall')
model.add_regressor(name='snow')

model.fit(train_1[['ds', 'y', 'temp_max', 'temp_min', 'rainfall', 'snow']])

future = model.make_future_dataframe(periods=7)
future = future.merge(test_1[['ds', 'temp_max', 'temp_min', 'rainfall', 'snow']], on='ds')
forecast_1 = model.predict(future)


10:38:52 - cmdstanpy - INFO - Chain [1] start processing
10:38:53 - cmdstanpy - INFO - Chain [1] done processing


In [63]:
eval = (pd.DataFrame(test_1[['ds', 'y']])
              .merge(forecast_1[['ds', 'yhat']], on='ds')
              )

eval.head()
np.sqrt(mean_squared_error(eval.y, eval.yhat))

0.376564034237821

In [64]:
eval.head()

,ds,y,yhat
0,2023-11-16,23.775000,23.860366
1,2023-11-17,25.033333,25.458914
2,2023-11-18,23.812500,24.021066
3,2023-11-19,24.333333,24.755379
4,2023-11-20,25.304167,24.657423


In [65]:
train_1 = pd.DataFrame(train_1.rename(columns={'date': 'ds', 'y': 'temp_mean'})).rename(columns={'temp_max': 'y'})
test_1 = pd.DataFrame(test_1.rename(columns={'date': 'ds', 'y': 'temp_mean'})).rename(columns={'temp_max': 'y'})

In [66]:
model = Prophet()
model.add_regressor(name='temp_mean')
model.add_regressor(name='temp_min')
model.add_regressor(name='rainfall')
model.add_regressor(name='snow')

model.fit(train_1[['ds', 'y', 'temp_mean', 'temp_min', 'rainfall', 'snow']])

future = model.make_future_dataframe(periods=7)
future = future.merge(test_1[['ds', 'temp_mean', 'temp_min', 'rainfall', 'snow']], on='ds')

forecast_1 = model.predict(future)

eval = (test_1
        .merge(forecast_1[['ds', 'yhat', 'yhat_lower','yhat_upper']], on='ds')
              )

np.sqrt(mean_squared_error(eval.y, eval.yhat))

10:39:44 - cmdstanpy - INFO - Chain [1] start processing
10:39:45 - cmdstanpy - INFO - Chain [1] done processing


0.5742560326199894

In [67]:
eval

,station,ds,y,temp_mean,temp_min,rainfall,snow,yhat,yhat_lower,yhat_upper
0,72202,2023-11-16,27.0,23.775000,21.0,True,False,27.123190,26.009474,28.065607
1,72202,2023-11-17,28.3,25.033333,23.0,True,False,27.816811,26.668746,28.825116
2,72202,2023-11-18,26.1,23.812500,22.0,True,False,26.547128,25.420279,27.591037
3,72202,2023-11-19,29.0,24.333333,21.0,False,False,28.039737,26.975185,29.071069
4,72202,2023-11-20,29.4,25.304167,20.6,False,False,29.809738,28.747841,30.854994
5,72202,2023-11-21,28.3,26.345833,24.4,False,False,29.033973,27.979343,30.055078
6,72202,2023-11-22,28.9,25.500000,23.3,False,False,28.419257,27.475474,29.479997


In [68]:
train_1 = pd.DataFrame(train_1.rename(columns={'date': 'ds', 'y': 'temp_max'})).rename(columns={'snow': 'y'})
test_1 = pd.DataFrame(test_1.rename(columns={'date': 'ds', 'y': 'temp_max'})).rename(columns={'snow': 'y'})


model = Prophet()
model.add_regressor(name='temp_mean')
model.add_regressor(name='temp_min')
model.add_regressor(name='rainfall')
model.add_regressor(name='temp_max')

model.fit(train_1[['ds', 'y', 'temp_mean', 'temp_min', 'rainfall', 'temp_max']])

future = model.make_future_dataframe(periods=7)
future = future.merge(test_1[['ds', 'temp_mean', 'temp_min', 'rainfall', 'temp_max']], on='ds')
forecast_1 = model.predict(future)
eval = (test_1
        .merge(forecast_1[['ds', 'yhat', 'yhat_lower','yhat_upper']], on='ds')
              )

np.sqrt(mean_squared_error(eval.y, eval.yhat))

0.0

## Fit a model on all stations?

In [70]:
train_data_pd.head()

,station,date,temp_max,temp_mean,temp_min,rainfall,snow
0,72202,2013-11-22,26.7,25.957143,24.4,True,False
1,72202,2013-11-23,27.2,25.033333,23.9,True,False
2,72202,2013-11-24,28.3,24.620833,21.7,False,False
3,72202,2013-11-25,25.6,23.179167,21.7,True,False
4,72202,2013-11-26,27.2,25.016667,22.8,True,False


In [72]:
train_df = pd.get_dummies(train_data_pd, columns=['station'])
test_df = pd.get_dummies(test_data_pd, columns=['station'])

In [78]:
train_df = pd.DataFrame(train_df.rename(columns={'date': 'ds', 'temp_min' : 'y'}))
test_df = pd.DataFrame(test_df.rename(columns={'date': 'ds',  'temp_min': 'y'}))

In [73]:
model = Prophet()
model.add_regressor(name='temp_mean')
model.add_regressor(name='snow')
model.add_regressor(name='rainfall')
model.add_regressor(name='temp_max')

In [75]:
for col in test_df.columns:
    if 'station' in col:
        model.add_regressor(name=col)

In [80]:

model.fit(train_df)

10:48:43 - cmdstanpy - INFO - Chain [1] start processing
10:49:36 - cmdstanpy - INFO - Chain [1] done processing


In [82]:
future = model.make_future_dataframe(periods=7)
future = future.merge(test_df.drop('y', axis=1), on='ds')

forecast_1 = model.predict(future)

In [83]:
eval_model = (test_df
        .merge(forecast_1[['ds', 'yhat']], on='ds')
              )

In [88]:
np.sqrt(mean_squared_error(eval_model.y, eval_model.yhat))

11.80841528875813

In [89]:
station_ids = daily_df.station.unique()

In [92]:
stations = dict(zip(range(len(station_ids)), station_ids))

In [96]:
models_min = dict()

In [102]:
def model_predictor(y: str, X: list, train: pd.DataFrame, 
                    test: pd.DataFrame, stations: dict, days: int = 7):
    
    models = dict()
    for i, station in stations.items():
        train_1 = train.loc[train.station == station]
        test_1 = test.loc[test.station == station]

        train_1 = pd.DataFrame(train_1.rename(columns={'date': 'ds', y: 'y'}))
        test_1 = pd.DataFrame(test_1.rename(columns={'date': 'ds', y: 'y'}))

        models[i] = Prophet()
        for x in X:
            models[i].add_regressor(name=x)

        models[i].fit(train_1[['ds', 'y'] + X])

        future = models[i].make_future_dataframe(periods=days)
        future = future.merge(test_1[['ds'] + X], on='ds') 

        forecast_1 = models[i].predict(future)

        eval_m = (pd.DataFrame(test_1[['ds', 'y']])
                    .merge(forecast_1[['ds', 'yhat']], on='ds')
                    )

        print(f"MSE = {np.sqrt(mean_squared_error(eval_m.y, eval_m.yhat))}")


    return models

In [103]:
model_min = model_predictor('temp_min', 
                            ['temp_max', 'temp_mean', 'rainfall', 'snow'], 
                            train_data_pd, test_data_pd, stations)

11:06:22 - cmdstanpy - INFO - Chain [1] start processing
11:06:22 - cmdstanpy - INFO - Chain [1] done processing
11:06:22 - cmdstanpy - INFO - Chain [1] start processing
11:06:23 - cmdstanpy - INFO - Chain [1] done processing
11:06:23 - cmdstanpy - INFO - Chain [1] start processing
11:06:23 - cmdstanpy - INFO - Chain [1] done processing
11:06:24 - cmdstanpy - INFO - Chain [1] start processing
11:06:25 - cmdstanpy - INFO - Chain [1] done processing
11:06:25 - cmdstanpy - INFO - Chain [1] start processing
11:06:25 - cmdstanpy - INFO - Chain [1] done processing
11:06:26 - cmdstanpy - INFO - Chain [1] start processing
11:06:26 - cmdstanpy - INFO - Chain [1] done processing
11:06:26 - cmdstanpy - INFO - Chain [1] start processing
11:06:27 - cmdstanpy - INFO - Chain [1] done processing
11:06:27 - cmdstanpy - INFO - Chain [1] start processing
11:06:28 - cmdstanpy - INFO - Chain [1] done processing
11:06:28 - cmdstanpy - INFO - Chain [1] start processing
11:06:28 - cmdstanpy - INFO - Chain [1]

MSE = 0.7022818047205439


In [97]:
for i, station in stations.items():
    train_1 = train_data_pd.loc[train_data_pd.station == station]
    test_1 = test_data_pd.loc[test_data_pd.station == station]

    train_1 = pd.DataFrame(train_1.rename(columns={'date': 'ds', 'temp_min': 'y'}))
    test_1 = pd.DataFrame(test_1.rename(columns={'date': 'ds', 'temp_min': 'y'}))

    models_min[i] = Prophet()
    models_min[i].add_regressor(name='temp_max')
    models_min[i].add_regressor(name='temp_mean')
    models_min[i].add_regressor(name='rainfall')
    models_min[i].add_regressor(name='snow')

    models_min[i].fit(train_1[['ds', 'y', 'temp_max', 'temp_mean', 'rainfall', 'snow']])

    future = models_min[i].make_future_dataframe(periods=7)
    future = future.merge(test_1[['ds', 'temp_max', 'temp_mean', 'rainfall', 'snow']], on='ds') 

    forecast_1 = models_min[i].predict(future)

    eval_m = (pd.DataFrame(test_1[['ds', 'y']])
                .merge(forecast_1[['ds', 'yhat']], on='ds')
                )
                        
    models_min[station] = np.sqrt(mean_squared_error(eval_m.y, eval_m.yhat))

10:57:38 - cmdstanpy - INFO - Chain [1] start processing
10:57:38 - cmdstanpy - INFO - Chain [1] done processing
10:57:39 - cmdstanpy - INFO - Chain [1] start processing
10:57:39 - cmdstanpy - INFO - Chain [1] done processing
10:57:39 - cmdstanpy - INFO - Chain [1] start processing
10:57:40 - cmdstanpy - INFO - Chain [1] done processing
10:57:40 - cmdstanpy - INFO - Chain [1] start processing
10:57:41 - cmdstanpy - INFO - Chain [1] done processing
10:57:41 - cmdstanpy - INFO - Chain [1] start processing
10:57:41 - cmdstanpy - INFO - Chain [1] done processing
10:57:42 - cmdstanpy - INFO - Chain [1] start processing
10:57:42 - cmdstanpy - INFO - Chain [1] done processing
10:57:42 - cmdstanpy - INFO - Chain [1] start processing
10:57:43 - cmdstanpy - INFO - Chain [1] done processing
10:57:43 - cmdstanpy - INFO - Chain [1] start processing
10:57:44 - cmdstanpy - INFO - Chain [1] done processing
10:57:44 - cmdstanpy - INFO - Chain [1] start processing
10:57:44 - cmdstanpy - INFO - Chain [1]

# Xgboost

In [10]:
def xgb_features(df, label=None, features =  None, lags = None):
    """
    Creates time series features
    """

    if not daily_df.date.dtype == 'datetime64[ns]' :
        df['date'] = pd.to_datetime(df['date'])
    
    
    df['month'] = df['date'].dt.month
    df['year'] = df['date'].dt.year
    df['dayofyear'] = df['date'].dt.dayofyear
    df['dayofmonth'] = df['date'].dt.day
    
    if lags:
        for i in range(1, lags+1):
            for  feat in features:
                df[feat + f'_lag_{i}'] = df[feat].shift(-i)
    if label:
        y = df[label]
        X = df.drop(label, axis=1)
        return X, y
    return df

In [122]:
xgb_data = xgb_features(daily_df)

In [126]:
xgb_data  = xgb_data.sort_values(by='date', ascending=True)
xgb_data['station'] = pd.Categorical(xgb_data['station'])
xgb_data = pd.get_dummies(xgb_data, columns=['station'])

In [16]:
def dict_configs(d):
    for vcomb in itertools.product(*d.values()):
        yield dict(zip(d.keys(), vcomb))

In [184]:
xgb_data.drop([y,'date'], axis=1).columns

Index(['station', 'temp_max', 'temp_mean', 'rainfall', 'snow', 'month', 'year',
       'dayofyear', 'dayofmonth'],
      dtype='object')

In [252]:
tss = TimeSeriesSplit(n_splits=3, test_size=4)


param_grid = {
    'max_depth': np.arange(2, 10, 2),
    'n_estimators': np.arange(100, 1200, 300),
}

params =  {'learning_rate': 0.1,
    'objective': 'reg:squarederror',
    'booster': 'gbtree',
    'n_jobs': 4,
    # 'enable_categorical': True,
    'random_state': 42}

fold = 0
# preds = []
scores = []
y = "temp_min"
xgb_data.dropna(inplace=True)
features = xgb_data.drop([y,'date'], axis=1).columns
grid = dict_configs(param_grid)
for train_idx, val_idx in tss.split(xgb_data):
    train = xgb_data.iloc[train_idx]
    test = xgb_data.iloc[val_idx]
    cv_results = dict()
    for cv_params in grid:
        X_train = train[features]
        y_train = train[y].shift(1).dropna()

        X_train.drop(index=X_train.index[:1], axis=0, inplace=True)

        X_test = test[features]
        y_test = test[y]

        params.update(cv_params)
        print(params)
        # reg = MultiOutputRegressor(xgb.XGBRegressor(**params))
        reg = xgb.XGBRegressor(**params)
        reg.fit(X_train, y_train,
                eval_set=[(X_train, y_train), (X_test, y_test)],
                verbose=0)

        y_pred = reg.predict(X_test)
#       preds.append(y_pred)
        cv_results[str(cv_params)] = np.sqrt(mean_squared_error(y_test, y_pred))
        scores.append(cv_results)

A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


{'learning_rate': 0.1, 'objective': 'reg:squarederror', 'booster': 'gbtree', 'n_jobs': 4, 'random_state': 42, 'max_depth': 2, 'n_estimators': 100}


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


{'learning_rate': 0.1, 'objective': 'reg:squarederror', 'booster': 'gbtree', 'n_jobs': 4, 'random_state': 42, 'max_depth': 2, 'n_estimators': 400}


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


...

{'learning_rate': 0.1, 'objective': 'reg:squarederror', 'booster': 'gbtree', 'n_jobs': 4, 'random_state': 42, 'max_depth': 8, 'n_estimators': 400}


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


{'learning_rate': 0.1, 'objective': 'reg:squarederror', 'booster': 'gbtree', 'n_jobs': 4, 'random_state': 42, 'max_depth': 8, 'n_estimators': 700}


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


{'learning_rate': 0.1, 'objective': 'reg:squarederror', 'booster': 'gbtree', 'n_jobs': 4, 'random_state': 42, 'max_depth': 8, 'n_estimators': 1000}


In [198]:
scoress

[{"{'max_depth': 2, 'n_estimators': 100}": 0.9848857801796105,
  "{'max_depth': 2, 'n_estimators': 200}": 0.9848857801796105,
  "{'max_depth': 2, 'n_estimators': 300}": 0.9848857801796105,
  "{'max_depth': 2, 'n_estimators': 400}": 0.9848857801796105,
  "{'max_depth': 2, 'n_estimators': 500}": 0.9848857801796105,
  "{'max_depth': 4, 'n_estimators': 100}": 0.9848857801796105,
  "{'max_depth': 4, 'n_estimators': 200}": 0.9848857801796105,
  "{'max_depth': 4, 'n_estimators': 300}": 0.9848857801796105,
  "{'max_depth': 4, 'n_estimators': 400}": 0.9848857801796105,
  "{'max_depth': 4, 'n_estimators': 500}": 0.9848857801796105,
  "{'max_depth': 6, 'n_estimators': 100}": 0.9848857801796105,
  "{'max_depth': 6, 'n_estimators': 200}": 0.9848857801796105,
  "{'max_depth': 6, 'n_estimators': 300}": 0.9848857801796105,
  "{'max_depth': 6, 'n_estimators': 400}": 0.9848857801796105,
  "{'max_depth': 6, 'n_estimators': 500}": 0.9848857801796105,
  "{'max_depth': 8, 'n_estimators': 100}": 0.9848857801

In [251]:
tss = TimeSeriesSplit(n_splits=3, test_size=4)

param_grid = {
    'max_depth': np.arange(2, 10, 2),
    'n_estimators': np.arange(100, 1200, 300),
}

params =  {'learning_rate': 0.1,
    'objective': 'reg:squarederror',
    'booster': 'gbtree',
    'n_jobs': 4,
    # 'enable_categorical': True,
    'random_state': 42}

fold = 0
# preds = []
scores = []
y = "temp_min"
xgb_data.dropna(inplace=True)
features = xgb_data.drop([y,'date'], axis=1).columns
grid = dict_configs(param_grid)
for train_idx, val_idx in tss.split(xgb_data):
    train = xgb_data.iloc[train_idx]
    test = xgb_data.iloc[val_idx]
    cv_results = dict()
    for cv_params in grid:
        X_train = train[features]
        y_train = train[y]

        y_train.to_frame().assign(
                day1 = y_train.shift(1),
                day2 = y_train.shift(2),
                day3= y_train.shift(3),
                day4 = y_train.shift(4)
            ).drop(y, axis=1).dropna()
        
        X_train.drop(index=X_train.index[:4], axis=0, inplace=True)

        X_test = test[features]
        y_test = test[y]
        y_test = y_test.to_numpy().reshape(-1, 4)

        params.update(cv_params)
        print(params)
        reg = xgb.XGBRegressor(**params)
        reg.fit(X_train, y_train,
                eval_set=[(X_train, y_train), (X_test, y_test)],
                verbose=0)

        y_pred = reg.predict(X_test)
#       preds.append(y_pred)
        cv_results[str(cv_params)] = np.sqrt(mean_squared_error(y_test, y_pred))
        scores.append(cv_results)

A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


{'learning_rate': 0.1, 'objective': 'reg:squarederror', 'booster': 'gbtree', 'n_jobs': 4, 'random_state': 42, 'max_depth': 2, 'n_estimators': 100}


XGBoostError: [15:11:21] /home/conda/feedstock_root/build_artifacts/xgboost-split_1700181168148/work/src/data/data.cc:455: Check failed: this->labels.Size() % this->num_row_ == 0 (4 vs. 0) : Incorrect size for labels.
Stack trace:
  [bt] (0) /home/marc/miniconda3/envs/604Final/lib/libxgboost.so(+0xb6361) [0x7f8b52ab6361]
  [bt] (1) /home/marc/miniconda3/envs/604Final/lib/libxgboost.so(xgboost::MetaInfo::SetInfoFromHost(xgboost::GenericParameter const&, xgboost::StringView, xgboost::Json)+0x2ac2) [0x7f8b52bb32c2]
  [bt] (2) /home/marc/miniconda3/envs/604Final/lib/libxgboost.so(xgboost::MetaInfo::SetInfo(xgboost::GenericParameter const&, xgboost::StringView, xgboost::StringView)+0x80) [0x7f8b52bb3490]
  [bt] (3) /home/marc/miniconda3/envs/604Final/lib/libxgboost.so(XGDMatrixSetInfoFromInterface+0xec) [0x7f8b52ab8f4c]
  [bt] (4) /home/marc/miniconda3/envs/604Final/lib/python3.10/lib-dynload/../../libffi.so.8(+0x6a4a) [0x7f8be2e36a4a]
  [bt] (5) /home/marc/miniconda3/envs/604Final/lib/pyth

#### Classificaition

In [ ]:
tss = TimeSeriesSplit(n_splits=3, test_size=4)


param_grid = {
    'max_depth': np.arange(2, 10, 2),
    'n_estimators': np.arange(100, 1200, 300),
}

params =  {'learning_rate': 0.1,
    'objective': 'reg:logistic',
    'booster': 'gbtree',
    'n_jobs': 4,
    # 'enable_categorical': True,
    'random_state': 42}

fold = 0
# preds = []
scores = []
y = "snow"
xgb_data.dropna(inplace=True)
features = xgb_data.drop([y,'date'], axis=1).columns
grid = dict_configs(param_grid)
for train_idx, val_idx in tss.split(xgb_data):
    train = xgb_data.iloc[train_idx]
    test = xgb_data.iloc[val_idx]
    cv_results = dict()
    for cv_params in grid:
        X_train = train[features]
        y_train = train[y].shift(1).dropna()

        X_train.drop(index=X_train.index[:1], axis=0, inplace=True)

        X_test = test[features]
        y_test = test[y]

        params.update(cv_params)
        print(params)
        # reg = MultiOutputRegressor(xgb.XGBRegressor(**params))
        reg = xgb.XGBRegressor(**params)
        reg.fit(X_train, y_train,
                eval_set=[(X_train, y_train), (X_test, y_test)],
                verbose=0)

        y_pred = reg.predict(X_test)
#       preds.append(y_pred)
        cv_results[str(cv_params)] = np.sqrt(mean_squared_error(y_test, y_pred))
        scores.append(cv_results)

# XGB Lagged Feature


In [11]:
daily_df.columns

Index(['station', 'date', 'temp_max', 'temp_mean', 'temp_min', 'rainfall',
       'snow'],
      dtype='object')

In [27]:
xgb_data = xgb_features(daily_df, features=['temp_max', 'temp_min','temp_mean', 'rainfall', 'snow'], lags=7)

In [28]:
xgb_data  = xgb_data.sort_values(by='date', ascending=True)
xgb_data['station'] = pd.Categorical(xgb_data['station'])
xgb_data = pd.get_dummies(xgb_data, columns=['station'])

In [25]:
xgb_data.set_index('date', inplace=True)

In [197]:
tss = TimeSeriesSplit(n_splits=3, test_size=100)


param_grid = {
    'max_depth': np.arange(2, 10, 2),
    'n_estimators': np.arange(100, 600, 100),
}

params =  {'learning_rate': 0.1,
    'objective': 'reg:logistic',
    'booster': 'gbtree',
    'n_jobs': 4,
    # 'enable_categorical': True,
    'random_state': 42}

fold = 0
# preds = []
scores = []
y = "snow"
xgb_data.dropna(inplace=True)
features = xgb_data.drop([y,'date'], axis=1).columns

for col in features:
    if "snow" in col or "rain" in col:
        xgb_data[col] = xgb_data[col].astype(bool)
        
grid = dict_configs(param_grid)
for train_idx, val_idx in tss.split(xgb_data):
    train = xgb_data.iloc[train_idx]
    test = xgb_data.iloc[val_idx]
    cv_results = dict()
    for cv_params in grid:
        X_train = train[features]
        y_train = train[y].shift(1).dropna().astype(bool)

        X_train.drop(index=X_train.index[:1], axis=0, inplace=True)

        X_test = test[features]
        y_test = test[y].astype(bool)

        params.update(cv_params)
        print(params)
        # reg = MultiOutputRegressor(xgb.XGBRegressor(**params))
        reg = xgb.XGBClassifier(**params)
        reg.fit(X_train, y_train,
                eval_set=[(X_train, y_train), (X_test, y_test)],
                verbose=0)

        y_pred = reg.predict(X_test)
#       preds.append(y_pred)
        cv_results[str(cv_params)] = np.sqrt(accuracy_score(y_test, y_pred))
        scores.append(cv_results)

## Using Mlforecast and XGBoost

In [65]:
from window_ops.rolling import rolling_mean, rolling_max, rolling_min

In [98]:
def dict_configs(d):
    for vcomb in itertools.product(*d.values()):
        yield dict(zip(d.keys(), vcomb))

In [110]:
test_date = '2023-11-16'
daily = True
daily_df["date"] = pd.to_datetime(daily_df["date"])
train_data_pd, test_data_pd = models.train_test_split(daily_df, test_date, daily)

In [111]:
train_data_pd.sort_values(by='date', ascending=True, inplace=True)
test_data_pd.sort_values(by='date', ascending=True, inplace=True)

A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [112]:
train_stations = train_data_pd.station
test_stations = test_data_pd.station

In [113]:
train_data_pd = pd.get_dummies(train_data_pd, columns=['station'])
test_data_pd  = pd.get_dummies(test_data_pd, columns=['station'])

In [114]:
static_feats = [feat for feat in train_data_pd.columns if 'station' in feat]

In [115]:
train_data_pd['station'] = train_stations
test_data_pd['station'] = test_stations

In [116]:
train_data_pd.dropna(inplace=True)
test_data_pd.dropna(inplace=True)

In [117]:
for feat in train_data_pd.columns:
    if 'snow' in feat or 'rain' in feat:
        train_data_pd[feat] = train_data_pd[feat].astype(int)
        test_data_pd[feat] = test_data_pd[feat].astype(int)

In [ ]:
xgb_data  = xgb_data.sort_values(by='date', ascending=True)
xgb_data['station'] = pd.Categorical(xgb_data['station'])
xgb_data = pd.get_dummies(xgb_data, columns=['station'])

In [119]:
model_list= [xgb.XGBClassifier(random_state=0,
                            max_depth = 10,
                            n_estimators=500,
                            learning_rate= 0.1,
                            objective =  'reg:logistic')]

fcst = mlforecast.MLForecast(models=model_list,
                   freq='D',
                   lags=[1,7,14],
                   lag_transforms={
                       1: [(rolling_mean, 7)],
                   },
                   date_features=['dayofweek', 'month', "year", "dayofyear"],
                   num_threads=6)



In [121]:
fcst.fit(train_data_pd, id_col='station',
         time_col='date',
         target_col='snow', 
         static_features=static_feats)

MLForecast(models=[XGBClassifier], freq=<Day>, lag_features=['lag1', 'lag7', 'lag14', 'rolling_mean_lag1_window_size7'], date_features=['dayofweek', 'month', 'year', 'dayofyear'], num_threads=6)

In [201]:
tem_min_preds

,station,date,XGBRegressor
0,72202,2023-11-16,21.022537
1,72202,2023-11-17,20.776085
2,72202,2023-11-18,21.061691
3,72202,2023-11-19,21.605604
4,72243,2023-11-16,14.101024
...,...,...,...
79,KYIP0,2023-11-19,3.119914
80,PALH0,2023-11-16,0.540359
81,PALH0,2023-11-17,-1.155002
82,PALH0,2023-11-18,-4.503466


In [200]:
results

,date,temp_max,temp_mean,temp_min,rainfall,snow,station,XGBClassifier
0,2023-11-16,27.0,23.775000,21.0,1,0,72202,0
1,2023-11-16,19.4,13.058333,8.3,0,0,KMIC0,0
2,2023-11-16,16.1,10.429167,5.0,0,0,74486,0
3,2023-11-16,9.0,4.566667,1.0,0,0,72793,0
4,2023-11-16,20.0,16.770833,14.4,1,0,72494,0
...,...,...,...,...,...,...,...,...
142,2023-11-22,20.4,14.329412,10.0,0,0,72290,0
143,2023-11-22,28.8,25.023529,23.5,1,0,91182,0
144,2023-11-22,13.0,9.952941,7.6,1,0,74486,0
145,2023-11-22,12.5,9.411765,6.7,0,0,72243,0


In [199]:
fcst.save('models/forecast.pkl')

AttributeError: 'MLForecast' object has no attribute 'save'

In [131]:
test_data_pd.drop(['station_72202', 'station_72243', 'station_72278', 'station_72290',
       'station_72327', 'station_72353', 'station_72405', 'station_72494',
       'station_72530', 'station_72565', 'station_72572', 'station_72606',
       'station_72681', 'station_72698', 'station_72793', 'station_74486',
       'station_91182', 'station_KARB0', 'station_KMIC0', 'station_KYIP0',
       'station_PALH0'], axis=1, inplace=True)

In [143]:
predictions = fcst.predict(h = 7, X_df = test_data_pd)

In [146]:
results = test_data_pd.merge(predictions, on = ['date', 'station'])

In [148]:
accuracy_score(results.snow, results.XGBClassifier)

0.9795918367346939

In [159]:
precision_score(results.snow, results.XGBClassifier)

1.0

In [160]:
f1_score(results.snow, results.XGBClassifier)

0.5714285714285715

In [161]:
recall_score(results.snow, results.XGBClassifier)

0.4

In [162]:
roc_auc_score(results.snow, results.XGBClassifier)

0.7

In [150]:
fcst.fit(train_data_pd, id_col='station',
         time_col='date',
         target_col='rainfall', 
         static_features=static_feats)

MLForecast(models=[XGBClassifier], freq=<Day>, lag_features=['lag1', 'lag7', 'lag14', 'rolling_mean_lag1_window_size7'], date_features=['dayofweek', 'month', 'year', 'dayofyear'], num_threads=6)

In [151]:
rainfall_preds = fcst.predict(h = 7, X_df = test_data_pd)

In [153]:
rainres = test_data_pd.merge(rainfall_preds, on = ['date', 'station'])

In [154]:
accuracy_score(rainres.rainfall, rainres.XGBClassifier)

0.7210884353741497

In [156]:
recall_score(rainres.rainfall, rainres.XGBClassifier)

0.48214285714285715

In [157]:
precision_score(rainres.rainfall, rainres.XGBClassifier)

0.6923076923076923

In [158]:
roc_auc_score(rainres.rainfall, rainres.XGBClassifier)

0.6751373626373627

In [167]:
cv_table = fcst.cross_validation(train_data_pd, 
                      id_col='station',n_windows=5,
                      h=4, time_col='date', 
                      target_col='rainfall',
                      static_features=static_feats)

In [182]:
def plot_cv(df, df_cv, uid, fname=None, last_n=4):
    cutoffs = df_cv.query('unique_id == @uid')['cutoff'].unique()
    print(cutoffs)
    fig, ax = plt.subplots(nrows=len(cutoffs), ncols=1, figsize=(14, 6), gridspec_kw=dict(hspace=0.8))
    for cutoff, axi in zip(cutoffs, ax.flat):
        df.query('unique_id == @uid').tail(last_n).set_index('ds').plot(ax=axi, title=uid, y='y')
        df_cv.query('unique_id == @uid & cutoff == @cutoff').set_index('ds').plot(ax=axi, title=uid, y='XGBClassifier')
    # fig.savefig(fname, bbox_inches='tight')
    plt.close()

In [183]:
cv_table

,station,date,cutoff,rainfall,XGBClassifier
0,PALH0,2023-10-27,2023-10-26,0,0
1,74486,2023-10-27,2023-10-26,0,0
2,72405,2023-10-27,2023-10-26,0,0
3,KARB0,2023-10-27,2023-10-26,1,1
4,91182,2023-10-27,2023-10-26,0,1
...,...,...,...,...,...
79,KARB0,2023-11-15,2023-11-11,0,0
80,72278,2023-11-15,2023-11-11,0,0
81,72290,2023-11-15,2023-11-11,1,0
82,KMIC0,2023-11-15,2023-11-11,0,0


In [185]:
plot_cv(test_data_pd.rename(columns={'date': 'ds', 'rainfall': 'y', 'station': 'unique_id'}),
        cv_table.rename(columns={'date': 'ds', 'rainfall': 'y', 'station': 'unique_id'}),
        uid = "PALH0")

<DatetimeArray>
['2023-10-26 00:00:00', '2023-10-30 00:00:00', '2023-11-03 00:00:00',
 '2023-11-07 00:00:00', '2023-11-11 00:00:00']
Length: 5, dtype: datetime64[ns]


### Using mlforecast with regression

In [188]:
model_list= [xgb.XGBRegressor(random_state=0,
                            max_depth = 10,
                            n_estimators=500,
                            learning_rate= 0.1,
                            objective =  'reg:squarederror')]

fcst = mlforecast.MLForecast(models=model_list,
                   freq='D',
                   lags=[1,7,14],
                   lag_transforms={
                       1: [(rolling_mean, 7)],
                   },
                   date_features=['dayofweek', 'month', "year", "dayofyear"],
                   num_threads=6)

In [189]:
fcst.fit(train_data_pd, id_col='station',
         time_col='date',
         target_col='temp_min', 
         max_horizon=4,
         static_features=static_feats)

MLForecast(models=[XGBRegressor], freq=<Day>, lag_features=['lag1', 'lag7', 'lag14', 'rolling_mean_lag1_window_size7'], date_features=['dayofweek', 'month', 'year', 'dayofyear'], num_threads=6)

In [190]:
tem_min_preds = fcst.predict(h = 4, X_df = test_data_pd)

In [194]:
results_min = test_data_pd.merge(tem_min_preds, on=['date', 'station'])

np.sqrt(mean_squared_error(results_min.temp_min, results_min.XGBRegressor))

3.3035091605687494

In [196]:
tem_min_preds.date.unique()

<DatetimeArray>
['2023-11-16 00:00:00', '2023-11-17 00:00:00', '2023-11-18 00:00:00',
 '2023-11-19 00:00:00']
Length: 4, dtype: datetime64[ns]